In [1]:
import os
import sys
from pathlib import Path # if you haven't already done so
file = Path(os.path.dirname(os.path.abspath("__file__"))).resolve()
parent, root = file.parent, file.parents[1]
sys.path.append(str(root))

import torch
from dtmgp.utils.sparse_grid.sgdesign import SparseGridDesign
from dtmgp.utils.sparse_grid.hyperbolic_cross import HyperbolicCrossDesign

In [2]:
# test Hyperbolic Cross Design
design_class = HyperbolicCrossDesign
print( design_class(dyadic_sort=True, indices_sort=True)(deg=3).points )
print( design_class(dyadic_sort=True, indices_sort=True)(deg=3).indices_sort )
print( design_class(dyadic_sort=True, indices_sort=True)(deg=3).sorted_vals_indices )

tensor([0.5000, 0.2500, 0.7500, 0.1250, 0.3750, 0.6250, 0.8750])
tensor([3, 1, 5, 0, 2, 4, 6])
(tensor([0.1250, 0.2500, 0.3750, 0.5000, 0.6250, 0.7500, 0.8750]), tensor([3, 1, 4, 0, 5, 2, 6]))


In [3]:
# initial setting
d = 2
eta = 4
input_bd = [[0,1]]*d # None
design_class = HyperbolicCrossDesign
dyadic_sort = True
indices_sort = True

# generate sparse grid design
sg = SparseGridDesign(d, eta, input_bd=input_bd, design_class=design_class).gen_sg(dyadic_sort=dyadic_sort, indices_sort=indices_sort)
x_tot = sg.pts_tot
id_prt = sg.id_prt # use self.tot_pts[ id_x_prt[t_sum, prt], : ] to extract grid points in each smolyak iter
pts_prt = sg.pts_prt # use self.pts_prt[t_sum, prt] to extract a d-dimensional list, each entry is one-dim points forming the sgdesign
pts_prt_set = sg.pts_prt_set

pts_set = sg.pts_set
n_pts = sg.n_pts

design_str_prt = sg.design_str_prt


indices_prt = sg.indices_prt
indices_prt_set = sg.indices_prt_set

indices_tot = sg.indices_tot
indices_set = sg.indices_set

In [54]:
design_str_prt[eta,2][1].points

tensor([0.5000])

In [10]:
pts_prt[eta,2]

[tensor([0.5000, 0.2500, 0.7500, 0.1250, 0.3750, 0.6250, 0.8750]),
 tensor([0.5000])]

In [11]:
indices_prt[eta,2]

[tensor([3, 1, 5, 0, 2, 4, 6]), tensor([3])]

In [7]:
print(indices_set)

indices_select = indices_prt_set[3,1]
print(indices_select)

tensor([[0, 3],
        [1, 1],
        [1, 3],
        [1, 5],
        [2, 3],
        [3, 0],
        [3, 1],
        [3, 2],
        [3, 3],
        [3, 4],
        [3, 5],
        [3, 6],
        [4, 3],
        [5, 1],
        [5, 3],
        [5, 5],
        [6, 3]])
tensor([[3, 3],
        [1, 3],
        [5, 3]])


In [40]:
list_full = [tuple(l) for l in indices_set.tolist()] # [n_u, d] size list of tuple
list_select = [tuple(l) for l in indices_select.tolist()] # [n_arrow, d] size list of tuple

index_dict = dict((value, idx) for idx,value in enumerate(list_full))
index_select = torch.tensor( [index_dict[x] for x in list_select] )
index_select

tensor([ 8,  2, 14])

In [18]:
i = torch.tensor([[0, 1, 2],
                   [1, 0, 2]])
v = torch.tensor([3, 4, 5], dtype=float)
sp_test = torch.sparse_coo_tensor(i, v, [3, 3])
sp_test.to_dense()

tensor([[0., 3., 0.],
        [4., 0., 0.],
        [0., 0., 5.]], dtype=torch.float64)

In [41]:
sp_id = sp_test._indices().T
print(sp_id)
print(index_select)

tensor([[0, 1],
        [1, 0],
        [2, 2]])
tensor([ 8,  2, 14])


In [46]:
index_select[sp_id]

tensor([[ 8,  2],
        [ 2,  8],
        [14, 14]])

In [47]:
sp_test._values()

tensor([3., 4., 5.], dtype=torch.float64)

In [48]:
sp_test_new = torch.sparse_coo_tensor(index_select[sp_test._indices()], sp_test._values(), [17, 17])

In [50]:
sp_test_new

tensor(indices=tensor([[ 8,  2, 14],
                       [ 2,  8, 14]]),
       values=tensor([3., 4., 5.]),
       size=(17, 17), nnz=3, dtype=torch.float64, layout=torch.sparse_coo)